In [21]:
import keras
from keras.layers import Input, Conv2D, Concatenate
from keras.models import Model
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint, CSVLogger, LambdaCallback
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from matplotlib import pyplot as plt
import keras.backend as K
import pandas as pd
import os
import numpy as np
import shutil
from PIL import Image
batch_size = 4
epochs = 400

In [22]:
df=pd.read_csv(r"placesData.csv")
datagen = ImageDataGenerator(rescale=1./255, validation_split=.2)
seed = 42
image_generator = datagen.flow_from_dataframe(df, 
                                      directory='.', 
                                      x_col='x_train', 
                                      y_col='y_train', 
                                      batch_size=batch_size,
                                      target_size=(512, 512),
                                      class_mode=None, 
                                      seed=seed)
mask_generator = datagen.flow_from_dataframe(df, 
                                     directory='.', 
                                     x_col='y_train', 
                                     y_col='x_train', # Or whatever 
                                     batch_size=batch_size,
                                     target_size=(512, 512),
                                     class_mode=None, 
                                     seed=seed)

train_generator = zip(image_generator, mask_generator)

Found 4000 validated image filenames.
Found 4000 validated image filenames.


In [23]:
mask = np.zeros((512,512,1))
for i in range(0, 512):
    for j in range(0, 512):
        if ((i-255)**2+(j-255)**2)**.5 > 32:
            mask[i,j,0] = ((i-255)**2+(j-255)**2)**.5
mask = (mask - mask.min()) / (mask.max() - mask.min())
print(mask[0][0][0])
print(mask[255][287][0])
print(mask[255][288][0])
inputMask = mask[None, :]
inputMask = np.tile(mask, (batch_size,1,1,1))
tMask = K.variable(inputMask)
inputMask = Input(tensor=tMask)
print(tMask.shape)

0.99609375
0.0
0.09115048351232839
(4, 512, 512, 1)


In [24]:
input = Input(shape=(512, 512, 3,), batch_shape=(batch_size, 512, 512, 3))
im = Concatenate(axis=-1)([input, inputMask])
print (im.shape)
l1 = Conv2D(filters=256, kernel_size=(16, 16), padding="same", activation='tanh')(im)
print(l1.shape)
l2 = Conv2D(filters=512, kernel_size=(8, 8), padding="same", activation='tanh')(l1)
print(l2.shape)
l3 = Conv2D(filters=512, kernel_size=(1, 1), padding="same", activation='tanh')(l2)
print(l3.shape)
output = Conv2D(filters=3, kernel_size=(8, 8), padding="same", activation='sigmoid')(l3)
print(output.shape)
model = Model(inputs=[input, inputMask], outputs=output)

(4, 512, 512, 4)
(4, 512, 512, 256)
(4, 512, 512, 512)
(4, 512, 512, 512)
(4, 512, 512, 3)


In [25]:
img = image.load_img("places/s00163.png", grayscale=False, target_size=(512, 512))
x = np.array(image.img_to_array(img))
print(x)
x = x[None, :]/255.
x = np.tile(x, (batch_size,1,1,1))
print(x.shape)

[[[155.  76.  10.]
  [161.  77.  13.]
  [176.  88.  24.]
  ...
  [ 92. 124. 109.]
  [ 80. 111.  96.]
  [ 63.  94.  79.]]

 [[159.  80.  14.]
  [171.  87.  23.]
  [175.  87.  23.]
  ...
  [ 55.  86.  71.]
  [ 99. 129. 117.]
  [106. 134. 122.]]

 [[153.  73.  10.]
  [168.  84.  20.]
  [180.  92.  28.]
  ...
  [ 75. 103.  91.]
  [ 94. 120. 109.]
  [ 83. 109.  98.]]

 ...

 [[165. 166. 135.]
  [ 72.  74.  50.]
  [109. 112.  95.]
  ...
  [ 22.  35.  17.]
  [ 22.  33.  16.]
  [ 44.  53.  36.]]

 [[148. 149. 117.]
  [ 84.  87.  60.]
  [115. 118.  99.]
  ...
  [ 58.  71.  53.]
  [ 58.  69.  52.]
  [ 89.  98.  79.]]

 [[125. 126.  92.]
  [119. 122.  95.]
  [ 88.  91.  72.]
  ...
  [ 49.  60.  43.]
  [ 37.  48.  31.]
  [ 43.  52.  33.]]]
(4, 512, 512, 3)


In [19]:
alg = optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.99999, amsgrad=True)
model.compile(optimizer=alg,
             loss='mean_squared_error',
             metrics=['accuracy', 'mse'])

In [ ]:
checkpoint = ModelCheckpoint("FGNchkpt.hdf5", monitor='loss', save_best_only=False, mode='auto', period=1, verbose=1)
csv_logger = CSVLogger('training.log')
image_saver = LambdaCallback(on_epoch_end=lambda epoch,logs: Image.fromarray((model.predict(x)[0]*255.).astype('uint8')).save("eImgs/"+str(epoch)+"p.jpeg"))
sanity_check1 = LambdaCallback(on_epoch_end=lambda epoch,logs: print((model.predict(x)[0]*255.)))
model.fit_generator(train_generator, steps_per_epoch=400/batch_size, epochs=epochs, shuffle=True, callbacks=[checkpoint, csv_logger, image_saver], verbose=1)
model.save('FGNweights.hdf5')

Epoch 1/400
100/100 [==============================] - 290s 3s/step - loss: 0.0431 - accuracy: 0.5989 - mse: 0.0431

Epoch 00001: saving model to FGNchkpt.hdf5
Epoch 2/400
100/100 [==============================] - 290s 3s/step - loss: 0.0099 - accuracy: 0.7868 - mse: 0.0099

Epoch 00002: saving model to FGNchkpt.hdf5
Epoch 3/400
100/100 [==============================] - 290s 3s/step - loss: 0.0090 - accuracy: 0.8179 - mse: 0.0090

Epoch 00003: saving model to FGNchkpt.hdf5
Epoch 4/400
100/100 [==============================] - 290s 3s/step - loss: 0.0093 - accuracy: 0.8084 - mse: 0.0093

Epoch 00004: saving model to FGNchkpt.hdf5
Epoch 5/400
100/100 [==============================] - 290s 3s/step - loss: 0.0097 - accuracy: 0.7926 - mse: 0.0097

Epoch 00005: saving model to FGNchkpt.hdf5
Epoch 6/400
100/100 [==============================] - 290s 3s/step - loss: 0.0096 - accuracy: 0.7861 - mse: 0.0096

Epoch 00006: saving model to FGNchkpt.hdf5
Epoch 7/400
100/100 [===================

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



100/100 [==============================] - 290s 3s/step - loss: 0.0067 - accuracy: 0.8128 - mse: 0.0067

Epoch 00176: saving model to FGNchkpt.hdf5
Epoch 177/400
100/100 [==============================] - 290s 3s/step - loss: 0.0065 - accuracy: 0.8198 - mse: 0.0065

Epoch 00177: saving model to FGNchkpt.hdf5
Epoch 178/400
100/100 [==============================] - 290s 3s/step - loss: 0.0067 - accuracy: 0.8205 - mse: 0.0067

Epoch 00178: saving model to FGNchkpt.hdf5
Epoch 179/400
100/100 [==============================] - 290s 3s/step - loss: 0.0063 - accuracy: 0.8198 - mse: 0.0063

Epoch 00179: saving model to FGNchkpt.hdf5
Epoch 180/400
100/100 [==============================] - 290s 3s/step - loss: 0.0067 - accuracy: 0.8154 - mse: 0.0067

Epoch 00180: saving model to FGNchkpt.hdf5
Epoch 181/400
100/100 [==============================] - 290s 3s/step - loss: 0.0069 - accuracy: 0.8130 - mse: 0.0069

Epoch 00181: saving model to FGNchkpt.hdf5
Epoch 182/400
100/100 [===================

In [26]:
model.load_weights("FGNchkptE273Sig.hdf5")

In [27]:
def save_image(model, path, tgtDir):
    img = image.load_img(path, grayscale=False, target_size=(512, 512))
    x = np.array(image.img_to_array(img))
    x = x[None, :]/255.
    x = np.tile(x, (batch_size,1,1,1))
    Image.fromarray((model.predict(x)[0]*255.).astype('uint8')).save(tgtDir+"/t"+path.replace('/', ' '))
save_image(model, "cac2-distorted.png", "test")

In [19]:
import glob
for imgPath in glob.glob("places/*"):
    save_image(model, imgPath, "test")

KeyboardInterrupt: 

In [ ]:
print(image_generator.next()[0][0][255])
print(mask_generator.next()[0][0][255])

In [56]:
img = image.load_img("placesMongrels/mongrolized_s00001_1.png", grayscale=False, target_size=(512, 512))
x = np.array(image.img_to_array(img))
img2 = image.load_img("placesMongrels/mongrolized_s00001_2.png", grayscale=False, target_size=(512, 512))
y = np.array(image.img_to_array(img2))
z = np.mean([x, y], axis=0)/255.
z = z[None:]
print(z.shape)
Image.fromarray(z*255).astype('uint8').save("t.png")

(512, 512, 3)


TypeError: Cannot handle this data type: (1, 1, 3), <f4